# Bayesian Machine Learning
## Challenges in Markov chain Monte Carlo for Bayesian neural networks

#### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from typing import Union
from tqdm import tqdm

sys.path.append('../source/')
from utils import effective_sample_size
from HMC import HMC
from MALA import MALA, AdaptiveMALA
from RWHM import Metropolis_Hastings
from model import Model
from utils import effective_sample_size, gelman_rubin

### Utils

In [2]:
def log_target_factory(
        mlp: Model,
        X: Union[np.ndarray, torch.Tensor],
        y: Union[np.ndarray, torch.Tensor],
    ):

    def log_target_fn(theta):
        log_target_tensor = mlp.compute_log_target(X, y, theta)
        log_target_array = log_target_tensor.cpu().detach().numpy()
        return log_target_array
        
    return log_target_fn

In [3]:
def grad_log_target_factory(
        mlp: Model,
        X: Union[np.ndarray, torch.Tensor],
        y: Union[np.ndarray, torch.Tensor],
    ):

    def grad_log_target_fn(theta):
        log_target_tensor = mlp.compute_log_target(X, y, theta)
        grad_tensor = mlp.compute_grad_log_target(log_target_tensor)
        grad_array = grad_tensor.cpu().detach().numpy()
        return grad_array

    return grad_log_target_fn

In [4]:
def diagnostic_MCMC(samples):
    ess = 0.0
    for i in range(samples.shape[0]):
        ess += effective_sample_size(samples[i])
    ess /= samples.shape[0]
    R_hat = gelman_rubin(samples)
    return ess, R_hat


In [5]:
def run(
    sizes,
    activations,
    X_train,
    y_train,
    N_steps=110_000,
    N_burnin=10_000,
    verbose=False,
    mh_params=dict(sigma_prop=None),
    hmc_params=dict(step_size=0.00004, n_leapfrog=10),
    mala_params=dict(step_size=None),
    sala_params=dict(step_size=None),
):
    samples_to_consider = (N_steps - N_burnin) // 10


    mlp = Model(sizes, activations)
    print(mlp)

    log_target_fn = log_target_factory(mlp, X_train, y_train)
    grad_log_target_fn = log_target_factory(mlp, X_train, y_train)

    N_params = mlp.num_parameters()
    mu = np.zeros(N_params)
    sigma =  10 * np.ones(N_params)
    def prior(size):
        return np.random.multivariate_normal(mean=mu, cov=np.diag(sigma), size=size)
    
    theta_0 = prior(1)[0]
    
    print("_Symmetric_Random_Walk_Metropolis_Hastings_".center(100).replace(" ", "=").replace("_", " "))
    MH_sampler = Metropolis_Hastings(
        log_target=log_target_fn, theta_0=theta_0, **mh_params
    )
    sample = np.zeros((5, N_steps-N_burnin+1, N_params))
    for i,seed in enumerate(range(40,45)):
        np.random.seed(seed)
        sample[i], _ = MH_sampler.sample(N_steps, N_burnin, verbose=verbose, return_burn_in=False)
        
    ess, R_hat = diagnostic_MCMC(sample)
    print(f"\tEffective sample size: {ess}")
    print(f"\tPSRF: {R_hat}")
    print("\n")
    
    print("_Hamiltonian_Monte_Carlo_".center(100).replace(" ", "=").replace("_", " "))
    HMC_sampler = HMC(
        log_target=log_target_fn, grad_log_target=grad_log_target_fn, theta_0=theta_0, **hmc_params
    )
    sample = np.zeros((5, N_steps-N_burnin+1, N_params))
    for i,seed in enumerate(range(40,45)):
        np.random.seed(seed)
        sample[i], _ = HMC_sampler.sample(N_steps, N_burnin, verbose=verbose, return_burn_in=False)
        
    ess, R_hat = diagnostic_MCMC(sample[:,samples_to_consider::, :])
    print(f"\tEffective sample size: {ess}")
    print(f"\tPSRF: {R_hat}")
    print("\n")

    
    print("_Adapative_MALA_".center(100).replace(" ", "=").replace("_", " "))
    SALA_sampler = AdaptiveMALA(
        log_target=log_target_fn, grad_log_target=grad_log_target_fn, theta_0=theta_0, **sala_params
    )
    sample = np.zeros((5, N_steps-N_burnin+1, N_params))
    for i,seed in enumerate(range(40,45)):
        np.random.seed(seed)
        sample[i], _ = SALA_sampler.sample(N_steps, N_burnin, verbose=verbose, return_burn_in=False)
        
    ess, R_hat = diagnostic_MCMC(sample)
    print(f"\tEffective sample size: {ess}")
    print(f"\tPSRF: {R_hat}")
    print("\n")
    

### Noisy XOR

In [6]:
X_noisy_xor_1 = pd.read_csv("../data/noisy_xor/data1/training/x.csv")
Y_noisy_xor_1 = pd.read_csv("../data/noisy_xor/data1/training/y.csv")
X_noisy_xor_1 = X_noisy_xor_1.to_numpy()
Y_noisy_xor_1 = Y_noisy_xor_1.to_numpy(dtype=int).flatten()
X_noisy_xor_1_test = pd.read_csv("../data/noisy_xor/data1/test/x.csv")
Y_noisy_xor_1_test = pd.read_csv("../data/noisy_xor/data1/test/y.csv")
X_noisy_xor_1_test = X_noisy_xor_1_test.to_numpy()
Y_noisy_xor_1_test = Y_noisy_xor_1_test.to_numpy(dtype=int).flatten()

In [7]:
run(
    sizes=[2, 2, 2],
    activations=[nn.ReLU(), nn.ReLU(), None],
    X_train=X_noisy_xor_1,
    y_train=Y_noisy_xor_1,
    N_steps=110_000,
    N_burnin=10_000,
)

Model(
  (layers): ModuleList(
    (0-1): 2 x Linear(in_features=2, out_features=2, bias=True)
  )
  (loss): CrossEntropyLoss()
)
============================ Symmetric Random Walk Metropolis Hastings =============================


100%|██████████| 110000/110000 [00:29<00:00, 3772.26it/s]


	Effective sample size: 240047.67360325475
	PSRF: (10.007678443847217+0j)


===================================== Hamiltonian Monte Carlo ======================================


100%|██████████| 110000/110000 [03:06<00:00, 588.44it/s]


	Effective sample size: 304402.21780226333
	PSRF: (5.041939788463842+0j)


========================================== Adapative MALA ==========================================


100%|██████████| 110000/110000 [00:57<00:00, 1900.71it/s]

	Effective sample size: 307765.59371596423
	PSRF: (3.2852045542382093+0j)




### Pima

In [8]:
X_pima_1 = pd.read_csv("../data/pima/data1/x.csv")
Y_pima_1 = pd.read_csv("../data/pima/data1/y.csv")
X_pima_1 = X_pima_1.to_numpy()
Y_pima_1 = Y_pima_1.to_numpy(dtype=int).flatten()
X_pima_1, X_pima_1_test, Y_pima_1, Y_pima_1_test = train_test_split(X_pima_1, Y_pima_1, test_size=0.4)

In [9]:
run(
    sizes=[8, 2, 2, 2],
    activations=[nn.ReLU()] * 3 + [None],
    X_train=X_pima_1,
    y_train=Y_pima_1,
    N_steps=110_000,
    N_burnin=10_000,
)

Model(
  (layers): ModuleList(
    (0): Linear(in_features=8, out_features=2, bias=True)
    (1-2): 2 x Linear(in_features=2, out_features=2, bias=True)
  )
  (loss): CrossEntropyLoss()
)
============================ Symmetric Random Walk Metropolis Hastings =============================


100%|██████████| 110000/110000 [00:30<00:00, 3632.83it/s]


	Effective sample size: 747596.3090287781
	PSRF: (29.288620469770812+0j)


===================================== Hamiltonian Monte Carlo ======================================


100%|██████████| 110000/110000 [03:17<00:00, 556.14it/s]


	Effective sample size: 566984.8125997509
	PSRF: (13.048020406290169+0j)


========================================== Adapative MALA ==========================================


100%|██████████| 110000/110000 [01:01<00:00, 1799.45it/s]


	Effective sample size: 627153.9496298257
	PSRF: (9.157149158667828+0j)




### Penguins

In [10]:
X_penguins = pd.read_csv("../data/penguins/x.csv")
X_penguins.drop(columns="year", inplace=True)
Y_penguins = pd.read_csv("../data/penguins/y.csv")
X_penguins = X_penguins.to_numpy()
Y_penguins = Y_penguins.to_numpy(dtype=int).flatten()
X_penguins, X_penguins_test, Y_penguins, Y_penguins_test = train_test_split(X_penguins, Y_penguins, test_size=0.2)

In [11]:
run(
    sizes=[6, 2, 2, 3],
    activations=[nn.ReLU()] * 3 + [None],
    X_train=X_penguins,
    y_train=Y_penguins,
    N_steps=110_000,
    N_burnin=10_000
)

Model(
  (layers): ModuleList(
    (0): Linear(in_features=6, out_features=2, bias=True)
    (1): Linear(in_features=2, out_features=2, bias=True)
    (2): Linear(in_features=2, out_features=3, bias=True)
  )
  (loss): CrossEntropyLoss()
)
============================ Symmetric Random Walk Metropolis Hastings =============================


100%|██████████| 110000/110000 [00:31<00:00, 3501.19it/s]


	Effective sample size: 646651.141004535
	PSRF: (381.141086796736+0j)


===================================== Hamiltonian Monte Carlo ======================================


100%|██████████| 110000/110000 [03:25<00:00, 535.47it/s]
/Users/benkabongo25/Studies/MVA/S2/Bayesian ML/Project/BayesianML-Project/notebooks/../source/utils.py:42: RuntimeWarning: invalid value encountered in scalar divide
  ess = ((det_variances / det_cov_matrix)**(1.0 / n_parameters)) * n_samples


	Effective sample size: nan
	PSRF: (5.699691338830044+0j)


========================================== Adapative MALA ==========================================


100%|██████████| 110000/110000 [01:02<00:00, 1762.61it/s]


	Effective sample size: 3416856.027302026
	PSRF: (2.759558005663408+0j)




### Hawks

In [12]:
X_hawks = pd.read_csv("../data/hawks/x.csv")
Y_hawks = pd.read_csv("../data/hawks/y.csv")
X_hawks = X_hawks.to_numpy()
Y_hawks = Y_hawks.to_numpy(dtype=int).flatten()
X_hawks, X_hawks_test, Y_hawks, Y_hawks_test = train_test_split(X_hawks, Y_hawks, test_size=0.2)

In [13]:
run(
    sizes=[6, 2, 2, 3],
    activations=[nn.ReLU()] * 3 + [None],
    X_train=X_penguins,
    y_train=Y_penguins,
    N_steps=110_000,
    N_burnin=10_000
)

Model(
  (layers): ModuleList(
    (0): Linear(in_features=6, out_features=2, bias=True)
    (1): Linear(in_features=2, out_features=2, bias=True)
    (2): Linear(in_features=2, out_features=3, bias=True)
  )
  (loss): CrossEntropyLoss()
)
============================ Symmetric Random Walk Metropolis Hastings =============================


100%|██████████| 110000/110000 [00:30<00:00, 3566.77it/s]


	Effective sample size: 684980.4230226666
	PSRF: (100.58523962787001+0j)


===================================== Hamiltonian Monte Carlo ======================================


100%|██████████| 110000/110000 [03:23<00:00, 539.30it/s]


	Effective sample size: 582743.2611454733
	PSRF: (9.638252912341281+0j)


========================================== Adapative MALA ==========================================


100%|██████████| 110000/110000 [01:03<00:00, 1723.84it/s]


	Effective sample size: 723552.259118391
	PSRF: (7.244244067368107+0j)


